In [1]:
def encode_dna(solution):
    dna = []
    remainder = abs(round(solution, 3)*1000)
    
    # Take each digit in the form: abc.def and put into list as ["f", "e", "d", ".", "c", "b", "a"]
    for i in range(6):
        part = remainder % 10**(i+1)
        digit = int(part * 10**(-i))
        dna.append(str(digit))
        if (i == 2):
            dna.append('.')
        remainder -= part
    
    # Reverse list order
    dna = dna[::-1]
    
    # Append '+' or '-' to list
    prefix = ['+']
    if (solution < 0):
        prefix = ['-']
    prefix.extend(dna)
    dna = prefix
    
    # Return list of form: ["s", "a", "b", "c", "d", "e", "f"]    (where s is the sign)
    return dna
    
def decode_dna(dna):
    return float("".join(dna))

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import sys

# Allows me to import my modules
sys.path.append('../../modules')

# The python definition of f(x) for problem 1.
from problem_function import f as fitness_function

In [3]:
# Test that I can encode and decode a value to get the same value out (to 6 SF & 3 DP).
dna = encode_dna(3.148787)
print("DNA: {}".format(dna))
solution = decode_dna(dna)
print("Solution: {}".format(solution))

DNA: ['+', '0', '0', '3', '.', '1', '4', '9']
Solution: 3.149


In [4]:
import random

# Default values for mutate_dna parameters
MUTATION_CHANCE = 0.1
SIGN_CHANGE_CHANCE = 0.05
sign_change_dictionary = {"-": "+", "+": "-"}

def mutate_dna(dna, chance = None, sign_change_chance = None):
    # Set defaults if parameter not provided
    chance = MUTATION_CHANCE if chance is None else chance
    sign_change_chance = SIGN_CHANGE_CHANCE if sign_change_chance is None else sign_change_chance
    
    # Chance of a change of sign
    if (random.random() <= sign_change_chance):
            dna[0] = sign_change_dictionary[dna[0]]
    
    # Chance of digit mutation has only a chance to occur
    if (random.random() <= chance):
                
        # Change of random digit 
        random_index = random.randrange(1, len(dna)-1)
        if (dna[random_index] != "."):
            dna[random_index] = str(random.randrange(0, 9))
            
    return dna

In [5]:
dna = encode_dna(200.000)
for i in range(100):
    dna = mutate_dna(dna, 0.05, 0.01)
    print(dna)


['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '0', '0', '0']
['+', '2', '0', '0', '.', '4', '0', '0']
['+', '2', '0', '0', '.', '4', '0', '0']
['+', '2', '0', '0', '.', '4', '0', '0']
['+', '2', '0', '0', '.', '4', '0', '0']
['+', '2', '0', '0', '.', '4', '0', '0']
['+', '2', '0', '0', '.', '4', '0', '0']
['+', '2', '0', '0', '.', '4', '0', '0']
['+', '2', '0', 

In [6]:
decode_dna(dna)

-383.4

In [7]:
def cross_over(dna1, dna2, cross_over_amount):
    resultant_dna = []
    templates = (dna1, dna2)
    current_template = random.randrange(0, 1)
    cross_over_points = set([random.randrange(0, 7) for i in range(cross_over_amount)])
    while (len(cross_over_points) != cross_over_amount):
        cross_over_points.add(random.randrange(0, 7))
    for i in range(8):
        if i in cross_over_points:
            current_template += 1
            current_template %= 2
        resultant_dna.append(templates[current_template][i])
        
    return resultant_dna

In [8]:
cross_over(encode_dna(111.111), encode_dna(-222.222), 4)

['-', '1', '2', '2', '.', '1', '1', '1']

In [9]:
class individual:
    def __init__(self, dna=None, gen=None):
        # If no dna is provided, generate dna from encoding a random float; with a random sign, exponent and significand
        if (dna == None):
            start_value = (2 * random.random() - 1) * 10**(random.randrange(0, 4))
            dna = encode_dna(start_value)
        self.dna = dna
        self.gen = 0 if gen is None else gen
    def get_dna(self):
        return self.dna
    def get_value(self):
        return decode_dna(self.dna)
    def get_gen(self):
        return self.gen

In [10]:
# Test of random individual instance values
for i in range(20):
    indiv = individual()
    print(indiv.get_value())

66.307
-17.2
58.398
3.791
-4.99
-7.967
-7.839
-9.819
-690.824
-0.535
4.68
-46.354
0.592
-36.546
65.58
-0.487
-3.567
997.983
-4.827
-736.057


In [11]:
# Default cross over amount
CROSS_OVER_AMOUNT = 2

def breed_individuals(ind1, ind2, cross_over_amount=None, mutation_chance=None, sign_change_chance=None):
    # Set defaults if parameter not provided
    cross_over_amount = CROSS_OVER_AMOUNT if cross_over_amount is None else cross_over_amount
    mutation_chance = MUTATION_CHANCE if mutation_chance is None else mutation_chance
    sign_change_chance = SIGN_CHANGE_CHANCE if sign_change_chance is None else sign_change_chance
    # Calculate the generation number of the offspring.
    gen1 = ind1.get_gen()
    gen2 = ind2.get_gen()
    next_gen = gen1+1 if gen1 > gen2 else gen2+1
    # Create the dna of the offspring
    dna1 = ind1.get_dna()
    dna2 = ind2.get_dna()
    new_dna = mutate_dna(cross_over(dna1, dna2, cross_over_amount), mutation_chance, sign_change_chance)
    return individual(new_dna, next_gen)

In [12]:
individual1 = individual(encode_dna(111.111), 1)
individual2 = individual(encode_dna(-222.222), 2)
new_individual = breed_individuals(individual1, individual2, 3, 1)

In [13]:
new_individual.get_value()

222.122

In [40]:
# Default GA parameters
# How many individuals per generation in the population.
POP_SIZE = 1000
# The amount of generations the GA will produce.
EPOCHS = 20
# What percentage of the population can be selected to breed.
FITNESS_UPPER_BOUND = 0.5
# How the parents are selected.
def selection_func(iteration, population, fitness_upper_bound):
    return (population[iteration], population[iteration+1])

MUTATION_CHANCE = 1
CROSS_OVER_AMOUNT = 3
SIGN_CHANGE_AMOUNT = 0.01

def genetic_algorithm(population_size, epochs, fitness_upper_bound, selection_function, cross_over_amount, mutation_chance, sign_change_chance, show_workings=False):
    # Initial population instantiation
    population = []
    for p in range(population_size):
        population.append(individual())

    # Genetic algorithm loop
    cumulative_population = []
    result = None
    for e in range(epochs):
        print("Initial epoch pop size: ", len(population))
        cumulative_population.extend(population)
        # Sort population by their values when input through the fitness function. Highest to lowest
        population.sort(key=lambda ind: fitness_function(ind.get_value()))
        if (show_workings):
            print("Generation {}'s, top 10 fittest individuals: {}".format(e+1, [population[i].get_value() for i in range(10)]))
        # For a percentage of the population select an amount of pairs of individuals
        for p in range(int(population_size*fitness_upper_bound/2)):
            individuals = selection_function(p, population, fitness_upper_bound)
            # Create an amount of next generation offspring with these individuals greater than or equal to the amount previous generation  
            for m in range(math.ceil(2/fitness_upper_bound)):
                population.append(breed_individuals(individuals[0], individuals[1], cross_over_amount, mutation_chance, sign_change_chance))
        print("Post breeding pop size: ", len(population))
        # Kill previous generation.
        population = population[population_size:]
        print("Post culling pop prev gen size: ", len(population))
        # Reduce the population size if its over the size parameter.
        while len(population) > population_size:
            population.pop()
        print("Post culling pop overflow size: ", len(population))
    # After all the epochs pick the first individual (fittest) in the population and obtain their value.
    solution = population[0].get_value()
    # Return the history of the population (cumulative population) and the solution. 
    return (cumulative_population, solution)

In [41]:
results = genetic_algorithm(POP_SIZE, EPOCHS, FITNESS_UPPER_BOUND, selection_func, CROSS_OVER_AMOUNT, MUTATION_CHANCE, SIGN_CHANGE_CHANCE, True)

Initial epoch pop size:  1000
Generation 1's, top 10 fittest individuals: [-0.001, 0.016, 0.019, 0.029, -0.038, -0.042, 0.05, 0.053, -0.055, 0.055]
Post breeding pop size:  2000
Post culling pop prev gen size:  1000
Post culling pop overflow size:  1000
Initial epoch pop size:  1000
Generation 2's, top 10 fittest individuals: [100.83, 100.812, 100.809, 100.636, 100.582, 100.491, 100.405, 0.0, 0.002, -0.004]
Post breeding pop size:  2000
Post culling pop prev gen size:  1000
Post culling pop overflow size:  1000
Initial epoch pop size:  1000
Generation 3's, top 10 fittest individuals: [100.836, 100.829, 100.812, 100.809, 101.405, 100.582, 100.491, 100.405, 100.405, 0.0]
Post breeding pop size:  2000
Post culling pop prev gen size:  1000
Post culling pop overflow size:  1000
Initial epoch pop size:  1000
Generation 4's, top 10 fittest individuals: [100.829, 100.812, 100.809, 101.205, 100.591, 100.491, 100.471, 100.465, 100.462, 100.415]
Post breeding pop size:  2000
Post culling pop prev